In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/AI Earth Hackathon/AI EarthHack Dataset.csv', encoding='latin-1')

In [ ]:
df.head()

,id,problem,solution
0,1,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m..."
1,2,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single..."
2,3,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa..."
3,4,The fashion industry is one of the top contrib...,The proposed solution is a garment rental serv...
4,5,The majority of the materials used in producin...,An innovative concept would be a modular elect...


In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=c50a5375785f26234c9848a71b6c40c568e4da1ce021c698681973b33923ad40
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
#remove stop words and punctuation
def remove_stop_punct(text):
    if isinstance(text, str):
        doc = nlp(text)
        return " ".join([token.text for token in doc if not token.is_stop and not token.is_punct])
    else:
        return text

In [ ]:
remove_stop_punct("Hello my name is AI EarthHack")

'Hello AI EarthHack'

In [ ]:
df["problem_processed"] = df["problem"].apply(remove_stop_punct)

In [ ]:
df["solution_processed"] = df["solution"].apply(remove_stop_punct)

In [ ]:
#compute similarity between problem and solution
from sklearn.metrics.pairwise import cosine_similarity
def compute_similarity(text1, text2):
  problem_solution_pair = [text1, text2]
  embeddings = model.encode(problem_solution_pair,convert_to_tensor=False)
  return cosine_similarity(embeddings)[0,1]

In [ ]:
compute_similarity("Hi my name is", "Hi I am called")

0.76778066

In [ ]:
df["problem"][0]

'The construction industry is indubitably one of the significant contributors to global waste, contributing approximately 1.3 billion tons of waste annually, exerting significant pressure on our landfills and natural resources. Traditional construction methods entail single-use designs that require frequent demolitions, leading to resource depletion and wastage.   '

In [ ]:
df["problem_processed"][0]

'construction industry indubitably significant contributors global waste contributing approximately 1.3 billion tons waste annually exerting significant pressure landfills natural resources Traditional construction methods entail single use designs require frequent demolitions leading resource depletion wastage   '

In [ ]:
#compute similarity between problem and solution for every pair and add to dictionary
similarities = []
for i in range(df.shape[0]):
  similarities.append(compute_similarity(df["problem_processed"][i], df["solution_processed"][i]))

In [ ]:
df["problem_solution_relevance"] = similarities

In [ ]:
df["problem"][0]

'The construction industry is indubitably one of the significant contributors to global waste, contributing approximately 1.3 billion tons of waste annually, exerting significant pressure on our landfills and natural resources. Traditional construction methods entail single-use designs that require frequent demolitions, leading to resource depletion and wastage.   '

In [ ]:
df["solution"][0]

"Herein, we propose an innovative approach to mitigate this problem: Modular Construction. This method embraces recycling and reuse, taking a significant stride towards a circular economy.   Modular construction involves utilizing engineered components in a manufacturing facility that are later assembled on-site. These components are designed for easy disassembling, enabling them to be reused in diverse projects, thus significantly reducing waste and conserving resources.  Not only does this method decrease construction waste by up to 90%, but it also decreases construction time by 30-50%, optimizing both environmental and financial efficiency. This reduction in time corresponds to substantial financial savings for businesses. Moreover, the modular approach allows greater flexibility, adapting to changing needs over time.  We believe, by adopting modular construction, the industry can transit from a 'take, make and dispose' model to a more sustainable 'reduce, reuse, and recycle' model

In [ ]:
scalability_words = ["scale", "expand", "increased demand", "scale-up", "scale up", "scalability"]

In [ ]:
#check for scability by see if how many of these words it has
scalability_scores = df["solution_processed"].apply(lambda x: sum(word in x.lower() for word in scalability_words) if isinstance(x, str) else 0).tolist()

In [ ]:
len(scalability_scores)

1300

In [ ]:
df["scalability_scores"] = scalability_scores

In [ ]:
df.head()

,id,problem,solution,problem_processed,solution_processed,problem_solution_relevance,scalability_scores
0,1,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m...",construction industry indubitably significant ...,propose innovative approach mitigate problem M...,0.449046,1
1,2,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single...",sure like feeling heat literally World Health ...,Imagine standing green hill single towering no...,0.457237,1
2,3,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa...",massive shift student learning digital platfor...,Implement Book Swap program educational instit...,0.487643,0
3,4,The fashion industry is one of the top contrib...,The proposed solution is a garment rental serv...,fashion industry contributors global pollution...,proposed solution garment rental service servi...,0.423262,1
4,5,The majority of the materials used in producin...,An innovative concept would be a modular elect...,majority materials producing electronic goods ...,innovative concept modular electronic device m...,0.506688,0


In [ ]:
max(scalability_scores)

3

In [ ]:
feasibility_words = ["feasible", "feasibility", "viable", "viability","practical", "implement", "implementable", "doable"]

In [ ]:
feasibility_scores = df["solution_processed"].apply(lambda x: sum(word in x.lower() for word in feasibility_words) if isinstance(x, str) else 0).tolist()

In [ ]:
len(feasibility_scores)

1300

In [ ]:
max(feasibility_scores)

4

In [ ]:
df["feasibility_scores"] = feasibility_scores

In [ ]:
impact_words = ["environmental impact", "financial impact", "efficiency", "productivity", "cost savings", "save cost", "save time", "time savings", "flexibility"]

In [ ]:
impact_scores = df["solution_processed"].apply(lambda x: sum(word in x.lower() for word in impact_words) if isinstance(x, str) else 0).tolist()

In [ ]:
len(impact_scores)

1300

In [ ]:
impact_scores[0:5]

[2, 0, 0, 0, 1]

In [ ]:
df["impact_scores"] = impact_scores

In [ ]:
market_potential_words = ["competitive advantage", "target audience", "market demand", "market share", "revenue", "sales", "profitability", "competitor","market expansion", "industry growth", "growth rate", "market trends", "expanding market", "market penetration", "unique selling proposition", "USP", "product differentiation", "technological advancements"]

In [ ]:
market_potential_scores = df["solution_processed"].apply(lambda x: sum(word in x.lower() for word in market_potential_words) if isinstance(x, str) else 0).tolist()

In [ ]:
max(market_potential_scores)

4

In [ ]:
len(market_potential_scores)

1300

In [ ]:
circular_economy_words = [
    "regenerate", "regeneration", "recycling", "waste reduction", "waste elimination",
    "pollution reduction", "circulation", "remanufacturing", "increased product lifespan",
    "zero waste", "resource recovery", "material reusability", "reusable", "lifespan", "eco-friendly", "renewable", "sustainable",
    "circular sourcing", "circular supply chain", "green", "environmental friendly",
    "sustainable", "sustainability", "reuse"
]

In [ ]:
circular_economy_scores = df["solution_processed"].apply(lambda x: sum(word in x.lower() for word in circular_economy_words) if isinstance(x, str) else 0).tolist()

In [ ]:
len(circular_economy_scores)

1300

In [ ]:
circular_economy_scores[0:5]

[4, 1, 1, 1, 1]

In [ ]:
max(circular_economy_scores)

9

In [ ]:
df['circular_economy_scores'] = circular_economy_scores
df['market_potential_scores'] = market_potential_scores

In [ ]:
maturity_words = [
    "Implemented",
    "Established",
    "Operational",
    "In practice",
    "Robust",
    "Proven track record",
    "Long-term success",
    "Extensive adoption",
    "Standardized",
    "Well-executed",
    "Optimized processes",
    "Consistent results",
    "Sustained impact",
    "Industry acceptance",
    "Matured model"
]

In [ ]:
maturity_scores = df["solution_processed"].apply(lambda x: sum(word in x.lower() for word in maturity_words) if isinstance(x, str) else 0).tolist()

In [ ]:
sum(maturity_scores)

0

In [ ]:
df_updated = df

In [ ]:
df_updated.head()

,id,problem,solution,problem_processed,solution_processed,problem_solution_relevance,scalability_scores,feasibility_scores,impact_scores,circular_economy_scores,market_potential_scores
0,1,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m...",construction industry indubitably significant ...,propose innovative approach mitigate problem M...,0.449046,1,1,2,4,0
1,2,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single...",sure like feeling heat literally World Health ...,Imagine standing green hill single towering no...,0.457237,1,0,0,1,0
2,3,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa...",massive shift student learning digital platfor...,Implement Book Swap program educational instit...,0.487643,0,0,0,1,0
3,4,The fashion industry is one of the top contrib...,The proposed solution is a garment rental serv...,fashion industry contributors global pollution...,proposed solution garment rental service servi...,0.423262,1,2,0,1,0
4,5,The majority of the materials used in producin...,An innovative concept would be a modular elect...,majority materials producing electronic goods ...,innovative concept modular electronic device m...,0.506688,0,1,1,1,1


In [ ]:
doc0 = nlp(df["solution_processed"][0])
[token.text for token in doc0 if token.pos_ == "NOUN"]

['approach',
 'mitigate',
 'problem',
 'method',
 'reuse',
 'economy',
 'construction',
 'components',
 'facility',
 'site',
 'components',
 'projects',
 'waste',
 'resources',
 'method',
 'decrease',
 'construction',
 'waste',
 'construction',
 'time',
 'efficiency',
 'reduction',
 'time',
 'savings',
 'businesses',
 'approach',
 'flexibility',
 'needs',
 'time',
 'construction',
 'industry',
 'transit',
 'model',
 'reuse',
 'recycle',
 'model',
 'industry',
 'feasibility',
 'concept',
 'markets',
 'globe',
 'scalability',
 'world',
 'application']

In [ ]:
[token.text for token in doc0.ents]

['Modular Construction', '90', '30 50']

In [ ]:
[token.text for token in nlp(df["solution_processed"][1]).ents]

['green hill', 'tomorrow']

In [ ]:
[token.text for token in nlp(df["solution_processed"][2]).ents]

[]

In [ ]:
[token.text for token in nlp(df["solution_processed"][3]).ents]

['month']

In [ ]:
def get_unique_nouns(solution_list):
    all_nouns = set()

    for solution in solution_list.tolist():
        if isinstance(solution, str):
            doc = nlp(solution.lower())
            noun_list = {token.text for token in doc if token.pos_ == "NOUN"}
            all_nouns.update(noun_list)

    return list(all_nouns)


In [ ]:
unique_nouns = get_unique_nouns(df_updated["solution_processed"])

In [ ]:
len(unique_nouns)

4654

In [ ]:
import pandas as pd

# Creating a DataFrame with 1300 rows and columns as unique nouns
df_unique_nouns = pd.DataFrame(0, index=range(1300), columns=unique_nouns)

df_unique_nouns.head()

,refunds,saturation,blush,nurture,directions,amounts,regulates,narrative,sodium,vapors,...,globe,grab,blockchain,retail,vermicomposting,production,cement,policies,reuse,economics
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for i in range(df_unique_nouns.shape[0]):
  if isinstance(df_updated["solution_processed"][i], str):
    for word in unique_nouns:
      if word in df_updated["solution_processed"][i]:
        df_unique_nouns[word][i] = 1

In [ ]:
df_unique_nouns.head()

,refunds,saturation,blush,nurture,directions,amounts,regulates,narrative,sodium,vapors,...,globe,grab,blockchain,retail,vermicomposting,production,cement,policies,reuse,economics
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [ ]:
"regulates" in df_updated["solution_processed"][0]

False

In [ ]:
frequencies = []
for noun in unique_nouns:
  frequencies.append(sum(df_unique_nouns[noun]))

In [ ]:
mean_value = np.mean(frequencies)

# Calculate median
median_value = np.median(frequencies)

# Calculate 25th percentile (Q1)
q1 = np.percentile(frequencies, 25)

# Calculate 75th percentile (Q3)
q3 = np.percentile(frequencies, 75)

# Calculate standard deviation
std_dev = np.std(frequencies)

# Display the results
print(f"Mean: {mean_value}")
print(f"Median: {median_value}")
print(f"25th Percentile (Q1): {q1}")
print(f"75th Percentile (Q3): {q3}")
print(f"Standard Deviation: {std_dev}")

Mean: 29.492909325311558
Median: 4.0
25th Percentile (Q1): 1.0
75th Percentile (Q3): 15.0
Standard Deviation: 97.94349868950457


In [ ]:
cols_to_drop = []
for noun in unique_nouns:
  if sum(df_unique_nouns[noun]) < 3:
    cols_to_drop.append(noun)

In [ ]:
df_unique_nouns_processed = df_unique_nouns.drop(cols_to_drop, axis = 1)

In [ ]:
df_unique_nouns_processed.shape

(1300, 2678)

In [ ]:
df_unique_nouns_processed.head()

,nurture,amounts,machine,reverse,movement,dam,hours,manages,craft,paradigm,...,turning,target,outlet,globe,blockchain,retail,production,cement,policies,reuse
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [ ]:
unique_boolean = []

In [ ]:
word_count_mapping = {}
for word in df_unique_nouns_processed.columns:
  word_count_mapping[word] = sum(df_unique_nouns_processed[word])

In [ ]:
max(word_count_mapping.values())

1293

In [ ]:
unique_counts = []
unique_rating = []
for i in range(df_unique_nouns_processed.shape[0]):
  count = 0
  for word in df_unique_nouns_processed.columns:
    if df_unique_nouns_processed[word][i] == 1 and word_count_mapping[word] <= 50:
      count += 1

  unique_counts.append(count)
  if count >= 40:
    unique_rating.append(3)
  elif count < 40 and count >= 30:
    unique_rating.append(2)
  else:
    unique_rating.append(1)





In [ ]:
unique_values, counts = np.unique(unique_counts, return_counts=True)

# Create a dictionary with values and their counts
value_counts_dict = dict(zip(unique_values, counts))

In [ ]:
sum(value_counts_dict.values())

1300

In [ ]:
value_counts_dict

{0: 23,
 1: 18,
 2: 17,
 3: 27,
 4: 24,
 5: 26,
 6: 28,
 7: 27,
 8: 35,
 9: 38,
 10: 29,
 11: 49,
 12: 25,
 13: 39,
 14: 45,
 15: 34,
 16: 45,
 17: 45,
 18: 40,
 19: 39,
 20: 42,
 21: 39,
 22: 36,
 23: 34,
 24: 36,
 25: 30,
 26: 40,
 27: 40,
 28: 31,
 29: 34,
 30: 30,
 31: 27,
 32: 26,
 33: 21,
 34: 26,
 35: 15,
 36: 13,
 37: 13,
 38: 5,
 39: 10,
 40: 16,
 41: 10,
 42: 10,
 43: 8,
 44: 1,
 45: 3,
 46: 4,
 47: 7,
 48: 1,
 49: 5,
 50: 3,
 51: 3,
 52: 3,
 53: 4,
 55: 4,
 56: 1,
 58: 2,
 60: 1,
 61: 1,
 62: 2,
 63: 1,
 64: 1,
 65: 1,
 67: 1,
 69: 1,
 71: 1,
 74: 1,
 88: 1,
 111: 1,
 115: 1}

In [ ]:
df_unique_nouns_processed['unique_rating'] = unique_rating

0       1
1       1
2       1
3       1
4       2
       ..
1295    1
1296    2
1297    1
1298    1
1299    3
Name: unique_rating, Length: 1300, dtype: int64

In [ ]:
df_updated_unique_rating = pd.concat([df_updated, df_unique_nouns_processed['unique_rating']], axis=1)

In [ ]:
df_updated_unique_rating.head()

,id,problem,solution,problem_processed,solution_processed,problem_solution_relevance,scalability_scores,feasibility_scores,impact_scores,circular_economy_scores,market_potential_scores,unique_rating
0,1,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m...",construction industry indubitably significant ...,propose innovative approach mitigate problem M...,0.449046,1,1,2,4,0,1
1,2,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single...",sure like feeling heat literally World Health ...,Imagine standing green hill single towering no...,0.457237,1,0,0,1,0,1
2,3,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa...",massive shift student learning digital platfor...,Implement Book Swap program educational instit...,0.487643,0,0,0,1,0,1
3,4,The fashion industry is one of the top contrib...,The proposed solution is a garment rental serv...,fashion industry contributors global pollution...,proposed solution garment rental service servi...,0.423262,1,2,0,1,0,1
4,5,The majority of the materials used in producin...,An innovative concept would be a modular elect...,majority materials producing electronic goods ...,innovative concept modular electronic device m...,0.506688,0,1,1,1,1,2


In [ ]:
num_classes = 3
#pd.qcut(df_unique_nouns_processed['unique_rating'], q=num_classes, labels=range(1, num_classes + 1)) #doesn't work with duplicate values


In [ ]:
list(set(df_updated_unique_rating['scalability_scores']))

[0, 1, 2, 3]

In [ ]:
unique_scalability_values, scalability_counts = np.unique(df_updated_unique_rating['scalability_scores'], return_counts=True)

# Create a dictionary with values and their counts
scalability_value_counts_dict = dict(zip(unique_scalability_values, scalability_counts))

In [ ]:
scalability_value_counts_dict

{0: 862, 1: 361, 2: 70, 3: 7}

In [ ]:
scalability_ratings = []
for val in df_updated_unique_rating['scalability_scores']:
  if val >= 2:
    scalability_ratings.append(3)
  elif val == 1:
    scalability_ratings.append(2)
  else:
    scalability_ratings.append(1)

In [ ]:
df_updated_unique_rating["scalability_ratings"] = scalability_ratings

In [ ]:
unique_relevance_values, relevance_counts = np.unique(df_updated_unique_rating['problem_solution_relevance'], return_counts=True)

# Create a dictionary with values and their counts
relevance_value_counts_dict = dict(zip(unique_relevance_values, relevance_counts))

In [ ]:
unique_relevance_ratings = []
for val in df_updated_unique_rating['problem_solution_relevance']:
  if val >= 0.6:
    unique_relevance_ratings.append(3)
  elif val >= 0.45:
    unique_relevance_ratings.append(2)
  else:
    unique_relevance_ratings.append(1)

In [ ]:
df_updated_unique_rating["relevance_ratings"] = unique_relevance_ratings

In [ ]:
df_updated_unique_rating.head()

,id,problem,solution,problem_processed,solution_processed,problem_solution_relevance,scalability_scores,feasibility_scores,impact_scores,circular_economy_scores,market_potential_scores,unique_rating,scalability_ratings,relevance_ratings
0,1,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m...",construction industry indubitably significant ...,propose innovative approach mitigate problem M...,0.449046,1,1,2,4,0,1,2,1
1,2,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single...",sure like feeling heat literally World Health ...,Imagine standing green hill single towering no...,0.457237,1,0,0,1,0,1,2,2
2,3,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa...",massive shift student learning digital platfor...,Implement Book Swap program educational instit...,0.487643,0,0,0,1,0,1,1,2
3,4,The fashion industry is one of the top contrib...,The proposed solution is a garment rental serv...,fashion industry contributors global pollution...,proposed solution garment rental service servi...,0.423262,1,2,0,1,0,1,2,1
4,5,The majority of the materials used in producin...,An innovative concept would be a modular elect...,majority materials producing electronic goods ...,innovative concept modular electronic device m...,0.506688,0,1,1,1,1,2,1,2


In [ ]:
df_updated_unique_rating.rename(columns={'scalability_ratings': 'scalability_rating', 'relevance_ratings':'relevance_rating'}, inplace=True)

In [ ]:
df_updated_unique_rating.head()

,id,problem,solution,problem_processed,solution_processed,problem_solution_relevance,scalability_scores,feasibility_scores,impact_scores,circular_economy_scores,market_potential_scores,unique_rating,scalability_rating,relevance_rating
0,1,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m...",construction industry indubitably significant ...,propose innovative approach mitigate problem M...,0.449046,1,1,2,4,0,1,2,1
1,2,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single...",sure like feeling heat literally World Health ...,Imagine standing green hill single towering no...,0.457237,1,0,0,1,0,1,2,2
2,3,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa...",massive shift student learning digital platfor...,Implement Book Swap program educational instit...,0.487643,0,0,0,1,0,1,1,2
3,4,The fashion industry is one of the top contrib...,The proposed solution is a garment rental serv...,fashion industry contributors global pollution...,proposed solution garment rental service servi...,0.423262,1,2,0,1,0,1,2,1
4,5,The majority of the materials used in producin...,An innovative concept would be a modular elect...,majority materials producing electronic goods ...,innovative concept modular electronic device m...,0.506688,0,1,1,1,1,2,1,2


In [ ]:
unique_feasibility_values, feasibility_counts = np.unique(df_updated_unique_rating['feasibility_scores'], return_counts=True)

# Create a dictionary with values and their counts
feasibility_value_counts_dict = dict(zip(unique_feasibility_values, feasibility_counts))

In [ ]:
feasibility_value_counts_dict

{0: 692, 1: 399, 2: 182, 3: 24, 4: 3}

In [ ]:
feasibility_ratings = []
for val in df_updated_unique_rating['feasibility_scores']:
  if val >= 2:
    feasibility_ratings.append(3)
  elif val == 1:
    feasibility_ratings.append(2)
  else:
    feasibility_ratings.append(1)

In [ ]:
df_updated_unique_rating["feasibility_rating"] = feasibility_ratings

In [ ]:
df_updated_unique_rating.head()

,id,problem,solution,problem_processed,solution_processed,problem_solution_relevance,scalability_scores,feasibility_scores,impact_scores,circular_economy_scores,market_potential_scores,unique_rating,scalability_rating,relevance_rating,feasibility_rating
0,1,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m...",construction industry indubitably significant ...,propose innovative approach mitigate problem M...,0.449046,1,1,2,4,0,1,2,1,2
1,2,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single...",sure like feeling heat literally World Health ...,Imagine standing green hill single towering no...,0.457237,1,0,0,1,0,1,2,2,1
2,3,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa...",massive shift student learning digital platfor...,Implement Book Swap program educational instit...,0.487643,0,0,0,1,0,1,1,2,1
3,4,The fashion industry is one of the top contrib...,The proposed solution is a garment rental serv...,fashion industry contributors global pollution...,proposed solution garment rental service servi...,0.423262,1,2,0,1,0,1,2,1,3
4,5,The majority of the materials used in producin...,An innovative concept would be a modular elect...,majority materials producing electronic goods ...,innovative concept modular electronic device m...,0.506688,0,1,1,1,1,2,1,2,2


In [ ]:
unique_impact_values, impact_counts = np.unique(df_updated_unique_rating['impact_scores'], return_counts=True)

# Create a dictionary with values and their counts
impact_value_counts_dict = dict(zip(unique_impact_values, impact_counts))

In [ ]:
impact_value_counts_dict

{0: 817, 1: 346, 2: 116, 3: 18, 4: 3}

In [ ]:
impact_ratings = []
for val in df_updated_unique_rating['impact_scores']:
  if val >= 2:
    impact_ratings.append(3)
  elif val == 1:
    impact_ratings.append(2)
  else:
    impact_ratings.append(1)

In [ ]:
df_updated_unique_rating["impact_rating"] = impact_ratings

In [ ]:
unique_ce_values, ce_counts = np.unique(df_updated_unique_rating['circular_economy_scores'], return_counts=True)

# Create a dictionary with values and their counts
ce_value_counts_dict = dict(zip(unique_ce_values, ce_counts))
ce_value_counts_dict

{0: 348, 1: 285, 2: 253, 3: 216, 4: 131, 5: 46, 6: 18, 7: 2, 9: 1}

In [ ]:
ce_ratings = []
for val in df_updated_unique_rating['circular_economy_scores']:
  if val >= 3:
    ce_ratings.append(3)
  elif val >= 1:
    ce_ratings.append(2)
  else:
    ce_ratings.append(1)

In [ ]:
df_updated_unique_rating["circular_economy_rating"] = ce_ratings

In [ ]:
unique_mp_values, mp_counts = np.unique(df_updated_unique_rating['market_potential_scores'], return_counts=True)

# Create a dictionary with values and their counts
mp_value_counts_dict = dict(zip(unique_mp_values, mp_counts))
mp_value_counts_dict

{0: 955, 1: 304, 2: 38, 3: 2, 4: 1}

In [ ]:
mp_ratings = []
for val in df_updated_unique_rating['market_potential_scores']:
  if val >= 2:
    mp_ratings.append(3)
  elif val == 1:
    mp_ratings.append(2)
  else:
    mp_ratings.append(1)

In [ ]:
df_updated_unique_rating["market_potential_rating"] = mp_ratings

In [ ]:
final_df = df_updated_unique_rating.drop(['problem_solution_relevance', 'scalability_scores', 'feasibility_scores', 'impact_scores', 'circular_economy_scores', 'market_potential_scores', 'problem_processed', 'solution_processed'], axis = 1)

In [ ]:
final_df.head()

,id,problem,solution,unique_rating,scalability_rating,relevance_rating,feasibility_rating,impact_rating,circular_economy_rating,market_potential_rating
0,1,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m...",1,2,1,2,3,3,1
1,2,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single...",1,2,2,1,1,2,1
2,3,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa...",1,1,2,1,1,2,1
3,4,The fashion industry is one of the top contrib...,The proposed solution is a garment rental serv...,1,2,1,3,1,2,1
4,5,The majority of the materials used in producin...,An innovative concept would be a modular elect...,2,1,2,2,2,2,2


In [ ]:
columns_to_average = ['unique_rating', 'scalability_rating', 'relevance_rating', 'feasibility_rating', 'impact_rating', 'circular_economy_rating', 'market_potential_rating']

# Create a new column 'average_rating' with the mean of selected columns
final_df['average_rating'] = final_df[columns_to_average].mean(axis=1)

,id,problem,solution,unique_rating,scalability_rating,relevance_rating,feasibility_rating,impact_rating,circular_economy_rating,market_potential_rating,average_rating
0,1,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m...",1,2,1,2,3,3,1,1.857143
1,2,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single...",1,2,2,1,1,2,1,1.428571
2,3,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa...",1,1,2,1,1,2,1,1.285714
3,4,The fashion industry is one of the top contrib...,The proposed solution is a garment rental serv...,1,2,1,3,1,2,1,1.571429
4,5,The majority of the materials used in producin...,An innovative concept would be a modular elect...,2,1,2,2,2,2,2,1.857143


In [ ]:
#sort by these columns
columns_to_sort_by = ['average_rating']

final_sorted_df = final_df.sort_values(by=columns_to_sort_by, ascending=False)

In [ ]:
final_sorted_df = final_sorted_df.reset_index()

In [ ]:
final_sorted_df = final_sorted_df.drop(['index', 'id'], axis = 1)

In [ ]:
final_sorted_df.head()

,problem,solution,unique_rating,scalability_rating,relevance_rating,feasibility_rating,impact_rating,circular_economy_rating,market_potential_rating,average_rating
0,Corporations creating e-waste are faced with t...,The improved solution involves TechLoopâs ev...,3,3,3,2,3,3,3,2.857143
1,Traditional methods of construction industry g...,The solution offered was deconstruction instea...,3,2,3,3,3,3,2,2.714286
2,The sustainability of public buildings is an i...,Implementing an ecologically advanced public b...,3,2,3,2,3,3,2,2.571429
3,The fashion industry uses a linear economic mo...,I propose creating a digital platform that con...,3,2,2,3,3,3,2,2.571429
4,Broad-based integration of the circular econom...,My solution utilizes AI to manage industrial e...,3,3,1,3,3,3,2,2.571429


In [ ]:
final_sorted_df["solution"][0]

"The improved solution involves TechLoopâ\x80\x99s evolution into a Circular Economy Business Suite. This suite's primary facet remains the digital platform for e-waste management, focusing on managing e-waste and reducing environmental impact. This platform would also analyze the lifecycle of the client corporation's electronic products to identify areas to extend their lifespan, reduce energy consumption, and foster a culture of sharing or leasing equipment rather than an outright purchase.  However, beyond managing e-waste, the Suite will offer consulting services to corporations to help them redesign their business processes and customer offerings around the principles of the circular economy, encouraging them to manufacture products that are meant to be re-used, re-sold, or recycled.   The Suite will also incorporate an e-waste marketplace, where corporations can list their e-waste items to be sold to refurbishing companies, recyclers, or even other corporations that may have a us

In [ ]:
final_sorted_df["solution"][4]

"My solution utilizes AI to manage industrial energy and material waste in 10 ways: predictive maintenance, waste sorting, smart inventory management, product design optimization, sustainable supply chain management, circular business models, energy management, sustainable packaging, water management, and autonomous waste collection.\n\n1.\tPredictive Maintenance: Predictive maintenance can be implemented in various industries, such as manufacturing and transportation. For example, a manufacturer could use AI-powered sensors to monitor the performance of its equipment and detect any abnormalities in real-time. If the system detects an issue, it can alert maintenance teams to take action before the equipment fails, reducing downtime and increasing the lifespan of the equipment.\n2.\tWaste Sorting: Waste sorting is a critical step in the circular economy, and AI can help optimize the process. For instance, an AI-powered robot could sort waste into different categories such as recyclable 

In [ ]:
final_sorted_df["problem"][4]

'Broad-based integration of the circular economy in various industries (& acceleration thereof)'

In [ ]:
final_sorted_df.to_csv("AI_Earth_Hackathon_problem_solution_sorted.csv")

In [ ]:
final_sorted_df.to_csv('/content/drive/My Drive/AI Earth Hackathon/AI_Earth_Hackathon_problem_solution_sorted.csv', index=False)

In [ ]:
df["solution"][1030]

nan

In [ ]:
df_unique_nouns_processed.head()

,nurture,amounts,machine,reverse,movement,dam,hours,manages,craft,paradigm,...,target,outlet,globe,blockchain,retail,production,cement,policies,reuse,unique_rating
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,2


In [ ]:
df_unique_nouns_processed.drop(['unique_rating'], axis = 1).to_csv('/content/drive/My Drive/AI Earth Hackathon/AI_Earth_Hackathon_unique_nouns_processed.csv', index=False)